# TP2 - Realidade Aumentada

__Aluno:__ Vinicius Silva Gomes

__Matrícula:__ 2021421869

In [1]:
# Importa as bibliotecas necessárias para o programa

import numpy as np
import cv2
import os

import matplotlib.pyplot as plt

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from PIL import Image

from objloader import *

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Extraindo frames do vídeo para calibração

Esse pequeno script foi feito para extrair os frames do vídeo e usá-los para calibrar a câmera e obter os parâmetros intrínsecos da câmera. O script, em suma, carrega o vídeo, usando o OpenCV, e salva cada frame no disco, em uma pasta chamada ./fames.

Dessa pasta, foram escolhidos 5 frames que apresentavam angulações, distâncias e rotações diferentes do tabuleiro xadrez e esses frames foram separados para serem usados na calibração da câmera. Esses frames escolhidos foram: __frame0.jpg, frame161.jpg, frame260.jpg, frame702.jpge e frame800.jpg__. 

__OBS: Para que as imagens sejam salvas, a pasta ./frames precisa ter sido criada previamente.__

In [7]:
# Extrai os frames do vídeo para realizar a calibração da câmera

# cam = cv2.VideoCapture("./entrada.mp4")

# current_frame = 0

# while(True):
#     ret, frame = cam.read()

#     if ret:
#         # Para esse trecho funcionar uma pasta ./frames deve existir no diretório do notebook
#         name = './frames/frame' + str(current_frame) + '.jpg'
  
#         cv2.imwrite(name, frame)
  
#         current_frame += 1
#     else:
#         break

# cam.release()
# cv2.destroyAllWindows()

## Obtendo os parâmetros intrínsecos e os coeficientes de distorção

Com os frames selecionados, foi usado o MatLAB para obter os parâmetros intrísecos da câmera. A opção "Camera Calibration" foi a escolhida. Os pontos do tabuleiro foram selecionados e após o mapeamento e as devidas funções internas do MatLAB terem sido executadas, a matriz de parâmetros intrínsecos foi obtida. Além disso, a partir dessa calibração os coeficientes de distorção também puderam ser obtidos.

A próxima célula apresenta a declaração dessa matriz com os dados de output do MatLAB.

<!-- OBS: A matriz foi transposta para se parecer com a matriz que foi estudada e usada como exemplo ao longo das aulas da disciplina. -->

In [3]:
# Matriz com os parâmetros intrínsecos da câmera
intrinsic_params = np.array(([410.556385620329, 0, 313.422287942592],
                             [0, 409.988094875811, 233.665786823660],
                             [0, 0, 1]), dtype="float32")

# Vetor com os coeficientes de distorção obtidos na calibração
distortion_coefficients = np.array([0.094979724334344, -0.258369409395469, 0, 0], dtype="float32")

print(intrinsic_params)
print(distortion_coefficients)

[[410.5564    0.      313.4223 ]
 [  0.      409.9881  233.66579]
 [  0.        0.        1.     ]]
[ 0.09497973 -0.25836942  0.          0.        ]


## Localizando os alvos ao longo do vídeo

As funções a seguir são responsáveis por localizar os alvos ao longo de um frame. O vídeo será executado e cada frame será passado como parâmetro para as funções para que os alvos sejam localizados nele e retornados de maneira adequada para serem exibidos pelo OpenGL.

In [4]:
def normalized_cross_correlation(target, template):
    target_normalized = (target - target.mean()) / np.std(target)
    template_normalized = (template - template.mean()) / np.std(template)
    
    return np.mean(target_normalized * template_normalized)

# Dado o possível alvo retificado, calcula sua semelhança com todas as possíveis rotações do
# template e retorna, primeiramente, se o possível alvo é de fato um alvo e qual a sua orientação, caso seja.
def find_orientation(rectified, template):
    # Rotaciona o template nas direções possíveis
    up = template.copy()
    right = cv2.rotate(template, cv2.ROTATE_90_CLOCKWISE)
    down = cv2.rotate(template, cv2.ROTATE_180)
    left = cv2.rotate(template, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    templates = [up, right, down, left]
    
    # Calcula o erro para cada uma delas
    err_up = normalized_cross_correlation(rectified, up)
    err_right = normalized_cross_correlation(rectified, right)    
    err_down = normalized_cross_correlation(rectified, down)
    err_left = normalized_cross_correlation(rectified, left)
    
    errors = [err_up, err_right, err_down, err_left]
    
    # Identifica o menor erro obtido
    min_error = max(errors)
    min_index = errors.index(min_error)
    
    # Caso o erro esteja no limiar estabelecido, o alvo foi identificado e terá sua orientação retornada
    # 0 -> cima, 1 -> direita, 2 -> baixo, 3 -> esquerda
    if min_error > 0.7 and min_error < 1:
        return True, min_index
    else:
        return False, -1
    
# Dado o frame com um possível alvo e as coordenadas do alvo, retifica a imagem para saber se é um alvo ou não 
def compare_target(thresh, target, copy):
    template = cv2.imread("./alvo.jpg", 0)
    
    # Cria a matriz com os pontos do template
    template_points = np.array([[0, 0],
                            [template.shape[0], 0],
                            [template.shape[0], template.shape[1]],
                            [0, template.shape[1]]])
    
    # Calcula a homografia do possível alvo em relação a matriz dos pontos do template
    homography, _ = cv2.findHomography(target, template_points)
    
    # Retifica o alvo com a matriz de homografia
    rectified = cv2.warpPerspective(thresh, homography, (template.shape[0], template.shape[1]))
        
    # Obtém se ele é um alvo e qual a orientação dele, caso seja
    is_target, orientation = find_orientation(rectified, template)
    
    return is_target, orientation
    
# Função principal que usa todas as outras para identificar os alvos em cada frame
def identify_targets(frame):
    copy = frame.copy()
    # Converte o frame para escala de tons de cinza
    gray_frame = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
        
    # Calcula a binarização do frame
    _, thresh = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)
        
    # Enfatiza as bordas presentes no frame binarizado
    edged = cv2.Canny(thresh, 80, 160)
    
    # Cria os contornos ao longo das bordas do frame
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
    target_contours = []
    targets = []

    # Para cada contorno identificado
    for contour in contours:
        # Aproxima sua forma geométrica para obter o número de lados
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.1 * perimeter, True)
        no_vertices = len(approx)
        
        # Caso o contorno identificado tenha 4 lados e seja convexo
        if no_vertices == 4:
            # Verifica se ele é um alvo
            is_target, orientation = compare_target(thresh, approx, copy)
                        
            # Caso seja um alvo, insere no vetor de alvos identificados naquele frame
            if is_target:
                target_contours.append(approx)
                targets.append((approx[:, 0][:].astype("float32"), orientation))
                
    # Desenha de verde os contornos identificados no frame
    cv2.drawContours(copy, target_contours, -1, (0, 255, 0), 2)
    
    # Retorna o frame contornado e os alvos identificados nesse frame
    return copy, targets

## Funções da OpenGL

In [5]:
 def initOpenGL(intrinsic_params, dimensions):
    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    fx = intrinsic_params[0, 0];
    fy = intrinsic_params[1, 1];
    
    fovy = 2 * np.arctan(0.5 * height/fy) * 180/np.pi;
    aspect = (width * fy)/(height * fx);
    near = 0.1;
    far = 100.0;
    
    gluPerspective(fovy, aspect, near, far);
    
def load_background(frame):
    background_id = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, background_id)
    
    background = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    
    height, width, channels = background.shape
    background = np.frombuffer(background.tobytes(), dtype=background.dtype, count = height * width * channels)    
    background.shape = (height, width, channels)

    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    
    return background_id, background
    
def place_background(frame, dimensions):
    background_id, background = load_background(frame)
    
    (width, height) = dimensions
    
    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)

    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, background_id)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glPushMatrix()

    glBegin(GL_QUADS)
    glTexCoord2i(0, 0); glVertex2i(0, 0)
    glTexCoord2i(1, 0); glVertex2i(width, 0)
    glTexCoord2i(1, 1); glVertex2i(width, height)
    glTexCoord2i(0, 1); glVertex2i(0, height)
    glEnd()
    
    glPopMatrix()

    glMatrixMode(GL_PROJECTION)
    glPopMatrix()

    glMatrixMode(GL_MODELVIEW)
    glDepthMask(GL_TRUE)
    glDisable(GL_TEXTURE_2D)
    glFlush()
    
def render_cube(length):
    glutWireCube(length);
    
def render_arrow(length, orientation):
    x_dir = 0
    y_dir = 0

    x_cone = 1
    y_cone = 0
    
    # Série de IF's que verificam a orientação do alvo e configuram os parâmetros
    # para translação e rotação da linha e cone que formam a flecha com direção do alvo
    if orientation == 0:
        x_dir = 0
        y_dir = -length
        x_cone = 1
        y_cone = 0
        
    if orientation == 1:
        x_dir = -length
        y_dir = 0
        x_cone = 0
        y_cone = -1
        
    if orientation == 2:
        x_dir = 0
        y_dir = length
        x_cone = -1
        y_cone = 0
        
    if orientation == 3:
        x_dir = length
        y_dir = 0
        x_cone = 0
        y_cone = 1
    
    glPushMatrix()
    
    # Desenha a linha da flecha com a direção do alvo
    glBegin(GL_LINES)
    glVertex3d(0, 0, 0)
    glVertex3d(x_dir, y_dir, 0)
    glEnd()
    
    # Desenha e rotaciona a ponta da flecha com a direção do alvo
    glTranslated(x_dir, y_dir, 0)
    glRotated(90, x_cone, y_cone, 0)
    glutWireCone(0.4, 1.4, 10, 10)
    
    glPopMatrix()
    
def render_3D_objects(obj, target_points, orientation, intrinsic_params, distortion_coefficients):
    # Pontos no mundo definidos de maneira arbitrária
    object_points = np.array([[1, -1, 0], [-1, -1, 0],
                              [-1, 1, 0], [1, 1, 0]], dtype="float32")
    
    # Calcula os vetores de rotação e translação
    _, rotation_vec, translation_vec = cv2.solvePnP(object_points, target_points, intrinsic_params, distortion_coefficients)
     
    # Converte o vetor de rotação para uma matriz de rotação 
    rotation_matrix = cv2.Rodrigues(rotation_vec)[0]
    
    # Monta a matriz de projeção
    # Os coeficientes da segunda e terceira linha foram negados para inverter os eixos Y e Z, para passar
    # do sistema de coordenadas da OpenCV para OpenGL
    projection_matrix = np.array([[rotation_matrix[0, 0], rotation_matrix[0, 1], rotation_matrix[0, 2], translation_vec[0]],
                                [-rotation_matrix[1, 0], -rotation_matrix[1, 1], -rotation_matrix[1, 2], -translation_vec[1]],
                                [-rotation_matrix[2, 0], -rotation_matrix[2, 1], -rotation_matrix[2, 2], -translation_vec[2]],
                                [0, 0, 0, 1]], dtype="float32")
    
    # Transpõe a matriz
    projection_matrix = projection_matrix.T
    
    glPushMatrix()
    
    # Carrega a matriz de projeção no OpenGL
    glLoadMatrixf(projection_matrix)
    
    # Renderiza o modelo do Pikachu
    glCallList(obj.gl_list)
    
    # Renderiza um cubo ao redor do alvo
    render_cube(3)
    
    # Renderiza a flecha que aponta para a direção do alvo
    render_arrow(3, orientation)
    
    glPopMatrix()
    
def displayCallback():
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glEnable(GL_TEXTURE_2D)
    
    # Carrega o objeto do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)

    ret, frame = video.read()

    if ret:
        frame_with_contour, targets = identify_targets(frame)
        place_background(frame_with_contour, (640, 480))

        for target in targets:
            target_coords, orientation = target
            render_3D_objects(obj, target_coords, orientation, intrinsic_params, distortion_coefficients)
            
        glutSwapBuffers()

def idleCallback():
    glutPostRedisplay()

## OpenGL Main

In [6]:
dimensions = (640, 480)

# Parâmetros intrínsecos e coeficientes de distorção obtidos com a calibração
intrinsic_params = np.array(([410.556385620329, 0, 313.422287942592],
                             [0, 409.988094875811, 233.665786823660],
                             [0, 0, 1]), dtype="float32")

# Vetor com os coeficientes de distorção obtidos na calibração
distortion_coefficients = np.array([0.094979724334344, -0.258369409395469, 0, 0], dtype="float32")

# Carrega o vídeo da pasta
video = cv2.VideoCapture("./entrada.mp4")

# Inicializa a glut
glutInit()
glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)

glutInitWindowSize(*dimensions)

# Cria a janela e inicializa o OpenGL com os parâmetros da câmera e as dimensões da janela
window = glutCreateWindow(b'TP2 - Realidade Aumentada - Vinicius Silva Gomes')
initOpenGL(intrinsic_params, dimensions)

# Chama as funções principais de display e entra no loop principal do OpenGL
glutDisplayFunc(displayCallback)
glutIdleFunc(idleCallback)

# Loop principal do OpenGL
glutMainLoop()

# Desaloca o vídeo de entrada
video.release()

/tmp/ipykernel_59919/1676604316.py:2: RuntimeWarning: invalid value encountered in true_divide
  target_normalized = (target - target.mean()) / np.std(target)
/tmp/ipykernel_59919/3363241205.py:137: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  projection_matrix = np.array([[rotation_matrix[0, 0], rotation_matrix[0, 1], rotation_matrix[0, 2], translation_vec[0]],
